 <div style="border: 2px solid black; padding: 10px; background-color: #f7f7f7; text-align: center;">
    <h1 style="color: Navy; text-decoration: underline;">TP4: Machine Learning II</h1>
</div>


#### Exercice 1 : Initialisation de l'environnement et des structures de données

In [1]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3")

state_size = env.observation_space.n  # 500
action_size = env.action_space.n      # 6

policy_table = np.ones((state_size, action_size)) / action_size

value_table = np.zeros(state_size)

print("Exemple de politique initiale :", policy_table[:5])
print("Exemple de valeurs initiales :", value_table[:5])


Exemple de politique initiale : [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
Exemple de valeurs initiales : [0. 0. 0. 0. 0.]


#### Exercice 2 : Exploration et collecte d'épisodes

In [2]:
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    print(f"\n🎮 Épisode {episode + 1}")
    
    while not done:
        action = env.action_space.sample()  
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        print(f"Action : {action} ➡ Récompense : {reward}")
        total_reward += reward
        state = next_state

    print(f"🔚 Récompense totale : {total_reward}")



🎮 Épisode 1
Action : 1 ➡ Récompense : -1
Action : 5 ➡ Récompense : -10
Action : 3 ➡ Récompense : -1
Action : 4 ➡ Récompense : -10
Action : 5 ➡ Récompense : -10
Action : 5 ➡ Récompense : -10
Action : 3 ➡ Récompense : -1
Action : 5 ➡ Récompense : -10
Action : 0 ➡ Récompense : -1
Action : 4 ➡ Récompense : -10
Action : 3 ➡ Récompense : -1
Action : 3 ➡ Récompense : -1
Action : 4 ➡ Récompense : -10
Action : 2 ➡ Récompense : -1
Action : 2 ➡ Récompense : -1
Action : 3 ➡ Récompense : -1
Action : 2 ➡ Récompense : -1
Action : 5 ➡ Récompense : -10
Action : 1 ➡ Récompense : -1
Action : 2 ➡ Récompense : -1
Action : 1 ➡ Récompense : -1
Action : 2 ➡ Récompense : -1
Action : 5 ➡ Récompense : -10
Action : 1 ➡ Récompense : -1
Action : 0 ➡ Récompense : -1
Action : 4 ➡ Récompense : -10
Action : 5 ➡ Récompense : -10
Action : 3 ➡ Récompense : -1
Action : 1 ➡ Récompense : -1
Action : 4 ➡ Récompense : -10
Action : 5 ➡ Récompense : -10
Action : 5 ➡ Récompense : -10
Action : 4 ➡ Récompense : -10
Action : 5 ➡ Ré

#### Exercice 3 : Mise à jour de la politique avec PPO

In [3]:
import numpy as np

gamma = 0.99
lr_policy = 0.1
clip_epsilon = 0.2

state, _ = env.reset()
done = False

episode_states = []
episode_actions = []
episode_rewards = []

while not done:
    action_probs = policy_table[state]
    action = np.random.choice(np.arange(action_size), p=action_probs)

    next_state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    episode_states.append(state)
    episode_actions.append(action)
    episode_rewards.append(reward)

    state = next_state

returns = []
G = 0
for r in reversed(episode_rewards):
    G = r + gamma * G
    returns.insert(0, G)

returns = np.array(returns)
returns = (returns - returns.mean()) / (returns.std() + 1e-8)  # Normalisation

for i in range(len(episode_states)):
    s = episode_states[i]
    a = episode_actions[i]
    advantage = returns[i] - value_table[s]
    
    old_prob = policy_table[s][a]
    
    ratio = old_prob / (old_prob + 1e-8)
    clipped_ratio = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon)
    update = min(ratio * advantage, clipped_ratio * advantage)

    policy_table[s][a] += lr_policy * update
    policy_table[s] /= np.sum(policy_table[s]) 

    value_table[s] += 0.1 * advantage


In [4]:
print("\n Mise à jour PPO sur un épisode")
print("-" * 40)
for i in range(len(episode_states)):
    s = episode_states[i]
    a = episode_actions[i]
    r = episode_rewards[i]
    v = value_table[s]
    adv = returns[i] - v
    prob = policy_table[s][a]

    print(f"État : {s}, Action : {a}, Récompense : {r}")
    print(f"  ➤ Valeur estimée V(s) : {v:.2f}")
    print(f"  ➤ Avantage A(s,a) : {adv:.2f}")
    print(f"  ➤ Ancienne probabilité : {prob:.2f}")
    print("-" * 40)



 Mise à jour PPO sur un épisode
----------------------------------------
État : 386, Action : 3, Récompense : -1
  ➤ Valeur estimée V(s) : -0.28
  ➤ Avantage A(s,a) : -1.28
  ➤ Ancienne probabilité : -0.13
----------------------------------------
État : 366, Action : 5, Récompense : -10
  ➤ Valeur estimée V(s) : -0.51
  ➤ Avantage A(s,a) : -1.08
  ➤ Ancienne probabilité : -0.18
----------------------------------------
État : 366, Action : 5, Récompense : -10
  ➤ Valeur estimée V(s) : -0.51
  ➤ Avantage A(s,a) : -0.99
  ➤ Ancienne probabilité : -0.18
----------------------------------------
État : 366, Action : 2, Récompense : -1
  ➤ Valeur estimée V(s) : -0.51
  ➤ Avantage A(s,a) : -0.90
  ➤ Ancienne probabilité : 0.15
----------------------------------------
État : 386, Action : 3, Récompense : -1
  ➤ Valeur estimée V(s) : -0.28
  ➤ Avantage A(s,a) : -1.15
  ➤ Ancienne probabilité : -0.13
----------------------------------------
État : 366, Action : 1, Récompense : -1
  ➤ Valeur esti

#### Évaluation de l'agent après entraînement

In [5]:
num_eval_episodes = 20
total_rewards = []

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = np.argmax(policy_table[state])
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward
        state = next_state

    total_rewards.append(total_reward)

print(" Moyenne des récompenses après entraînement :", np.mean(total_rewards))


 Moyenne des récompenses après entraînement : -289.55
